In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train= pd.read_csv('/kaggle/input/regularization-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/regularization-techniques/test.csv')

In [ ]:
train_values=df_train.values
test_values = df_test.values

In [ ]:
X_train = np.array(train_values[:,1:])
Y_train = np.array(train_values[:,0])
X_test = np.array(test_values[:,1:])
Y_test = np.array(test_values[:,0])

**Data Augumentation (Rescaling)**

In [ ]:
X_train=X_train.reshape(-1,28,28,1)/255.0
X_train.shape

In [ ]:
X_test = X_test.reshape(-1,28,28,1)/255.0
X_test.shape

**Importing Libraries**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical 


In [ ]:
number_of_labels = np.max(Y_train)
number_of_labels

 **Model without regularization techniques**

In [ ]:
def create_model(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model
    

In [ ]:
X_train[0].shape

In [ ]:
model=create_model(X_train[0].shape)

In [ ]:
to_categorical(Y_train,number_of_labels+1).shape

In [ ]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

In [ ]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model without regularization: "+ str(accuracy))

L1 Regularization

In [ ]:
from keras import regularizers

In [ ]:
def create_model_with_L1(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l2(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model=create_model_with_L1(X_train[0].shape)

In [ ]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

In [ ]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L1 Regularization: "+ str(accuracy))

***L2 Regularization with Dropout***

In [ ]:
def create_model_with_L2_Dropout(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l2(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model=create_model_with_L2_Dropout(X_train[0].shape)

In [ ]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

In [ ]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L2 regularization and Dropout: "+ str(accuracy))

**Model with L1,L2,Dropout and EarlyStopping**

In [ ]:
def create_model_with_L1_L2_Dropout_EarlyStopping(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l1(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model=create_model_with_L1_L2_Dropout_EarlyStopping(X_train[0].shape)

In [ ]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r'C:\Label.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [ ]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),
          batch_size=32,epochs=10,
          validation_split=0.2,
          callbacks=callbacks)

In [ ]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L2 , L1 and Dropout with earlyStopping and ReduceLrOnPlateau: "+ str(accuracy))

In [ ]:
X_tra=X_train.reshape(-1,784)
X_tes=X_test.reshape(-1,784)

**AdaBoost Classifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(X_tra,Y_train)


In [ ]:
model.score(X_tes,Y_test)

**XGBoost**

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()
model_xgb.fit(X_tra, Y_train)
model_xgb.score(X_tes,Y_test)